<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gradient-Descent-구현하기" data-toc-modified-id="Gradient-Descent-구현하기-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Gradient Descent 구현하기</a></span><ul class="toc-item"><li><span><a href="#(Batch)-Gradient-Descent" data-toc-modified-id="(Batch)-Gradient-Descent-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>(Batch) Gradient Descent</a></span></li><li><span><a href="#SGD(Stochastic-GD)-구현" data-toc-modified-id="SGD(Stochastic-GD)-구현-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>SGD(Stochastic GD) 구현</a></span></li><li><span><a href="#Mini-batch-GD-구현" data-toc-modified-id="Mini-batch-GD-구현-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Mini-batch GD 구현</a></span></li><li><span><a href="#Keras에서-구현되는-Mini-batch-GD--구현" data-toc-modified-id="Keras에서-구현되는-Mini-batch-GD--구현-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Keras에서 구현되는 Mini-batch GD  구현</a></span></li></ul></li></ul></div>

# Gradient Descent 구현하기
- numpy로 구현
- 활용할 Loss function: ``MSE``
- 사용 데이터: Boston 주택 가격 데이터

In [3]:
import numpy as np
import pandas as pd

# 보스턴 데이터 로드
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
bostonDF.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## (Batch) Gradient Descent
- Gradient를 계산하는 데 모든 데이터 사용하는 방법

In [6]:
# 2개의 변수만 사용
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate):
    N = len(target)
    
    # 예측값
    pred = (w1*rm) + (w2*lstat) + bias
    # error
    error = target - pred
    # bias도 업데이트 수식을 np.dot으로 만들기 위해 bias값 임의로 생성
    bias_factors = np.ones((N,))
    
    # update -> Gradient 수식은 MSE 손실함수 기반임
    w1_update = -learning_rate*(2/N)*(np.dot(rm.T, error))
    w2_update = -learning_rate*(2/N)*(np.dot(lstat.T, error))
    bias_update = -learning_rate*(2/N)*(np.dot(bias_factors.T, error))
    
    # MSE 계산
    mse = np.mean(np.square(error))
    
    return w1_update, w2_update, bias_update, mse

In [17]:
# 파라미터 초기화 후 Gradient Descent 수행
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # 파라미터 초기화
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # Epoch 마다 Gradient Descent 수행
    for i in range(iter_epochs):
        # Update 계산
        w1_update, w2_update, bias_update, mse_loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        
        # 파라미터 업데이트
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
            
    return w1, w2, mse_loss    

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])


final_w1, final_w2, mse_loss = gradient_descent(scaled_features,
                                               bostonDF['PRICE'],
                                               iter_epochs=1000,
                                               verbose=True)

In [ ]:
# Keras 이용해서 간단하게 FC Layer 만들기
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # Output 노드개수 = 1, 인풋 데이터 feature가 2개,  kernel=weight
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])

model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(x=scaled_features, y=bostonDF['PRICE'].values, epochs=1000)
prediction = model.predict(scaled_features)

## SGD(Stochastic GD) 구현
- SGD는 전체 데이터 중 하나의 데이터만 가지고 Gradient를 계산

In [29]:
def get_udpate_weights_value_sgd(bias, w1, w2, rm, lstat, target, learning_rate):
    N = 1
    
    pred = (w1*rm) + (w2*lstat) + bias
    error = target - pred
    bias_factors = np.ones((N,))
    
    w1_update = -learning_rate*(2/N)*(np.dot(rm.T, error))
    w2_update = -learning_rate*(2/N)*(np.dot(lstat.T, error))
    bias_update = -learning_rate*(2/N)*(np.dot(bias_factors.T, error))
    
    return w1_update, w2_update, bias_update

In [32]:
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # 파라미터 초기화
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    
    # 하나의 데이터 랜덤으로 샘플링할 랜덤 시드
    np.random.seed = 42
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    for i in range(iter_epochs):
        sample_idx = np.random.choice(target.shape[0], 1)
        rm_sgd = rm[sample_idx]
        lstat_sgd = lstat[sample_idx]
        target_sgd = target[sample_idx]
        
        # 파라미터 업데이트
        w1_update, w2_update, bias_update = get_udpate_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        
        if verbose:
            print('Epoch:', i+1, '/', iter_epochs)
            pred = (w1*rm) + (w2*lstat) + bias
            error = target - pred
            mse = np.mean(np.square(error))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'MSE:', mse)
            
    return w1, w2, bias, mse 

In [ ]:
final_w1, fianl_w2, final_bias, mse = st_gradient_descent(scaled_features,
                                                         bostonDF['PRICE'],
                                                         iter_epochs=1000,
                                                         verbose=True)

## Mini-batch GD 구현
- 이론적인 Mini-batch GD는 한 epoch를 수행할 때마다 랜덤하게 batch_size만큼 데이터를 가져와서 한 번의 Gradient를 계산

In [34]:
def get_update_weights_value_batch(bias, w1, w2, rm, lstat, target, learning_rate):
    N = len(target)
    
    pred = (w1*rm) + (w2*lstat) + bias
    error = target - pred
    bias_factors = np.ones((N,))
    
    # update
    w1_update = -learning_rate*(2/N)*np.dot(rm.T, error)
    w2_update = -learning_rate*(2/N)*np.dot(lstat.T, error)
    bias_update = -learning_rate*(2/N)*np.dot(bias_factors.T, error)
    
    return w1_update, w2_update, bias_update

In [35]:
def mini_batch_sgd(features, target, iter_epochs=1000, batch_size=32, verbose=True):
    # 파라미터 초기화
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    
    # Mini-batch 샘플링
    np.random.seed = 42
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    
    # epoch
    for i in range(iter_epochs):
        batch_idx = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_idx]
        lstat_batch = lstat[batch_idx]
        target_batch = target[batch_idx]
        
        # 파라미터 업데이트
        w1_update, w2_update, bias_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        
        if verbose:
            print('Epoch:', i+1, '/', iter_epochs)
            pred = (w1*rm) + (w2*lstat) + bias
            error = target - pred
            mse = np.mean(np.square(error))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'MSE:', mse)
    
    return w1, w2, bias, mse
        

In [ ]:
final_w1, final_w2, final_bias, mse = mini_batch_sgd(scaled_features,
                                                    bostonDF['PRICE'])

## Keras에서 구현되는 Mini-batch GD  구현
- 한 epoch를 batch_size로 전체 데이터셋을 다 학습하면 그 때 1회 epoch로 카운트하는 방식으로 학습
- 보통 딥러닝 프레임워크들이 이런식으로 구현 for 학습되는 데이터의 균형 확보 위해

In [37]:
def keras_min_batch_sgd(features, target, iter_epochs=1000, batch_size=32, verbose=True):
    # 파라미터 초기화
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1,))
    
    rm = features[:, 0]
    lstat = features[:, 1]
    learning_rate = 0.01
    
    # epoch
    for i in range(iter_epochs):
        # Batch size만큼 반복해서 전체 데이터셋 학습
        for step in range(0, target.shape[0], batch_size):
            rm_batch = rm[step: step+batch_size]
            lstat_batch = lstat[step: step+batch_size]
            target_batch = target[step: step+batch_size]
            
            # 파라미터 업데이트
            w1_update, w2_update, bias_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
            
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
            
            if verbose:
                print('Epoch:', i+1, '/', iter_epochs)
                pred = (w1*rm) + (w2*lstat) + bias
                error = target - pred
                mse = np.mean(np.square(error))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'MSE:', mse)
                
    return w1, w2, bias, mse
    

In [ ]:
final_w1, final_w2, final_bias, mse = keras_min_batch_sgd(scaled_features,
                                                         bostonDF['PRICE'])

In [40]:
# 미니배치 GD를 Keras로 수행
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(1, input_shape=(2,), activation=None, kernel_initializer='zeros', bias_initializer='zeros')
])

model.compile(optimizer=Adam(learning_rate=0.01), loss='mse',
             metrics=['mse'])
model.fit(x=scaled_features, y=bostonDF['PRICE'], batch_size=64, epochs=1000)
pred = model.predict(scaled_features)

Epoch 1/1000
506/506 [==============================] - 0s 102us/sample - loss: 589.3550 - mean_squared_error: 589.3550
Epoch 2/1000
506/506 [==============================] - 0s 22us/sample - loss: 582.9411 - mean_squared_error: 582.9410
Epoch 3/1000
506/506 [==============================] - 0s 23us/sample - loss: 576.5058 - mean_squared_error: 576.5058
Epoch 4/1000
506/506 [==============================] - 0s 18us/sample - loss: 570.1817 - mean_squared_error: 570.1817
Epoch 5/1000
506/506 [==============================] - 0s 19us/sample - loss: 563.9319 - mean_squared_error: 563.9319
Epoch 6/1000
506/506 [==============================] - 0s 23us/sample - loss: 557.6997 - mean_squared_error: 557.6997
Epoch 7/1000
506/506 [==============================] - 0s 17us/sample - loss: 551.5325 - mean_squared_error: 551.5325
Epoch 8/1000
506/506 [==============================] - 0s 26us/sample - loss: 545.4508 - mean_squared_error: 545.4508
Epoch 9/1000
506/506 [=========================

506/506 [==============================] - 0s 18us/sample - loss: 275.6985 - mean_squared_error: 275.6985
Epoch 70/1000
506/506 [==============================] - 0s 17us/sample - loss: 272.6782 - mean_squared_error: 272.6783
Epoch 71/1000
506/506 [==============================] - 0s 18us/sample - loss: 269.7593 - mean_squared_error: 269.7593
Epoch 72/1000
506/506 [==============================] - 0s 21us/sample - loss: 266.8312 - mean_squared_error: 266.8312
Epoch 73/1000
506/506 [==============================] - 0s 23us/sample - loss: 263.9573 - mean_squared_error: 263.9573
Epoch 74/1000
506/506 [==============================] - 0s 31us/sample - loss: 261.0953 - mean_squared_error: 261.0953
Epoch 75/1000
506/506 [==============================] - 0s 37us/sample - loss: 258.3157 - mean_squared_error: 258.3157
Epoch 76/1000
506/506 [==============================] - 0s 24us/sample - loss: 255.5594 - mean_squared_error: 255.5594
Epoch 77/1000
506/506 [==============================]

506/506 [==============================] - 0s 16us/sample - loss: 143.5571 - mean_squared_error: 143.5571
Epoch 138/1000
506/506 [==============================] - 0s 16us/sample - loss: 142.4373 - mean_squared_error: 142.4373
Epoch 139/1000
506/506 [==============================] - 0s 17us/sample - loss: 141.3846 - mean_squared_error: 141.3846
Epoch 140/1000
506/506 [==============================] - 0s 16us/sample - loss: 140.2986 - mean_squared_error: 140.2986
Epoch 141/1000
506/506 [==============================] - 0s 17us/sample - loss: 139.2618 - mean_squared_error: 139.2618
Epoch 142/1000
506/506 [==============================] - 0s 15us/sample - loss: 138.2420 - mean_squared_error: 138.2420
Epoch 143/1000
506/506 [==============================] - 0s 16us/sample - loss: 137.2135 - mean_squared_error: 137.2135
Epoch 144/1000
506/506 [==============================] - 0s 16us/sample - loss: 136.2292 - mean_squared_error: 136.2292
Epoch 145/1000
506/506 [=======================

506/506 [==============================] - 0s 22us/sample - loss: 98.4538 - mean_squared_error: 98.4538
Epoch 205/1000
506/506 [==============================] - 0s 21us/sample - loss: 98.0792 - mean_squared_error: 98.0792
Epoch 206/1000
506/506 [==============================] - 0s 22us/sample - loss: 97.7204 - mean_squared_error: 97.7204
Epoch 207/1000
506/506 [==============================] - 0s 20us/sample - loss: 97.3583 - mean_squared_error: 97.3583
Epoch 208/1000
506/506 [==============================] - 0s 17us/sample - loss: 96.9767 - mean_squared_error: 96.9767
Epoch 209/1000
506/506 [==============================] - 0s 19us/sample - loss: 96.6306 - mean_squared_error: 96.6306
Epoch 210/1000
506/506 [==============================] - 0s 19us/sample - loss: 96.2622 - mean_squared_error: 96.2622
Epoch 211/1000
506/506 [==============================] - 0s 18us/sample - loss: 95.9163 - mean_squared_error: 95.9163
Epoch 212/1000
506/506 [==============================] - 0s 21

Epoch 273/1000
506/506 [==============================] - 0s 17us/sample - loss: 79.7989 - mean_squared_error: 79.7990
Epoch 274/1000
506/506 [==============================] - 0s 17us/sample - loss: 79.6001 - mean_squared_error: 79.6001
Epoch 275/1000
506/506 [==============================] - 0s 20us/sample - loss: 79.3853 - mean_squared_error: 79.3853
Epoch 276/1000
506/506 [==============================] - 0s 19us/sample - loss: 79.1840 - mean_squared_error: 79.1840
Epoch 277/1000
506/506 [==============================] - 0s 18us/sample - loss: 78.9785 - mean_squared_error: 78.9785
Epoch 278/1000
506/506 [==============================] - 0s 18us/sample - loss: 78.7698 - mean_squared_error: 78.7698
Epoch 279/1000
506/506 [==============================] - 0s 18us/sample - loss: 78.5629 - mean_squared_error: 78.5629
Epoch 280/1000
506/506 [==============================] - 0s 17us/sample - loss: 78.3622 - mean_squared_error: 78.3622
Epoch 281/1000
506/506 [========================

...

Epoch 684/1000
506/506 [==============================] - 0s 15us/sample - loss: 33.6909 - mean_squared_error: 33.6909
Epoch 685/1000
506/506 [==============================] - 0s 16us/sample - loss: 33.6552 - mean_squared_error: 33.6552
Epoch 686/1000
506/506 [==============================] - 0s 17us/sample - loss: 33.6222 - mean_squared_error: 33.6222
Epoch 687/1000
506/506 [==============================] - 0s 16us/sample - loss: 33.5938 - mean_squared_error: 33.5938
Epoch 688/1000
506/506 [==============================] - 0s 15us/sample - loss: 33.5586 - mean_squared_error: 33.5586
Epoch 689/1000
506/506 [==============================] - 0s 18us/sample - loss: 33.5225 - mean_squared_error: 33.5225
Epoch 690/1000
506/506 [==============================] - 0s 18us/sample - loss: 33.4939 - mean_squared_error: 33.4939
Epoch 691/1000
506/506 [==============================] - 0s 16us/sample - loss: 33.4619 - mean_squared_error: 33.4619
Epoch 692/1000
506/506 [========================

506/506 [==============================] - 0s 16us/sample - loss: 31.9585 - mean_squared_error: 31.9585
Epoch 753/1000
506/506 [==============================] - 0s 17us/sample - loss: 31.9389 - mean_squared_error: 31.9389
Epoch 754/1000
506/506 [==============================] - 0s 17us/sample - loss: 31.9231 - mean_squared_error: 31.9231
Epoch 755/1000
506/506 [==============================] - 0s 18us/sample - loss: 31.9020 - mean_squared_error: 31.9020
Epoch 756/1000
506/506 [==============================] - 0s 17us/sample - loss: 31.8871 - mean_squared_error: 31.8871
Epoch 757/1000
506/506 [==============================] - 0s 15us/sample - loss: 31.8679 - mean_squared_error: 31.8679
Epoch 758/1000
506/506 [==============================] - 0s 17us/sample - loss: 31.8518 - mean_squared_error: 31.8518
Epoch 759/1000
506/506 [==============================] - 0s 16us/sample - loss: 31.8375 - mean_squared_error: 31.8375
Epoch 760/1000
506/506 [==============================] - 0s 15

Epoch 821/1000
506/506 [==============================] - 0s 17us/sample - loss: 31.0701 - mean_squared_error: 31.0701
Epoch 822/1000
506/506 [==============================] - 0s 19us/sample - loss: 31.0620 - mean_squared_error: 31.0620
Epoch 823/1000
506/506 [==============================] - 0s 16us/sample - loss: 31.0567 - mean_squared_error: 31.0567
Epoch 824/1000
506/506 [==============================] - 0s 18us/sample - loss: 31.0474 - mean_squared_error: 31.0474
Epoch 825/1000
506/506 [==============================] - 0s 19us/sample - loss: 31.0403 - mean_squared_error: 31.0403
Epoch 826/1000
506/506 [==============================] - 0s 18us/sample - loss: 31.0283 - mean_squared_error: 31.0283
Epoch 827/1000
506/506 [==============================] - 0s 19us/sample - loss: 31.0221 - mean_squared_error: 31.0221
Epoch 828/1000
506/506 [==============================] - 0s 17us/sample - loss: 31.0148 - mean_squared_error: 31.0148
Epoch 829/1000
506/506 [========================

506/506 [==============================] - 0s 22us/sample - loss: 30.7004 - mean_squared_error: 30.7004
Epoch 890/1000
506/506 [==============================] - 0s 22us/sample - loss: 30.6981 - mean_squared_error: 30.6981
Epoch 891/1000
506/506 [==============================] - 0s 19us/sample - loss: 30.6967 - mean_squared_error: 30.6967
Epoch 892/1000
506/506 [==============================] - 0s 21us/sample - loss: 30.6920 - mean_squared_error: 30.6920
Epoch 893/1000
506/506 [==============================] - 0s 19us/sample - loss: 30.6911 - mean_squared_error: 30.6911
Epoch 894/1000
506/506 [==============================] - 0s 16us/sample - loss: 30.6865 - mean_squared_error: 30.6865
Epoch 895/1000
506/506 [==============================] - 0s 19us/sample - loss: 30.6831 - mean_squared_error: 30.6831
Epoch 896/1000
506/506 [==============================] - 0s 20us/sample - loss: 30.6791 - mean_squared_error: 30.6791
Epoch 897/1000
506/506 [==============================] - 0s 23

Epoch 958/1000
506/506 [==============================] - 0s 21us/sample - loss: 30.5683 - mean_squared_error: 30.5683
Epoch 959/1000
506/506 [==============================] - 0s 24us/sample - loss: 30.5684 - mean_squared_error: 30.5684
Epoch 960/1000
506/506 [==============================] - 0s 23us/sample - loss: 30.5676 - mean_squared_error: 30.5676
Epoch 961/1000
506/506 [==============================] - 0s 21us/sample - loss: 30.5657 - mean_squared_error: 30.5657
Epoch 962/1000
506/506 [==============================] - 0s 21us/sample - loss: 30.5654 - mean_squared_error: 30.5654
Epoch 963/1000
506/506 [==============================] - 0s 21us/sample - loss: 30.5639 - mean_squared_error: 30.5639
Epoch 964/1000
506/506 [==============================] - 0s 22us/sample - loss: 30.5636 - mean_squared_error: 30.5636
Epoch 965/1000
506/506 [==============================] - 0s 19us/sample - loss: 30.5624 - mean_squared_error: 30.5624
Epoch 966/1000
506/506 [========================